In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# Set the working directory
os.chdir("C:\TradeGroup")
my_wd = os.getcwd()
new_directory = my_wd

def create_folder(folder_path):
    if not os.path.exists(f'{new_directory}/{folder_path}'):
        os.makedirs(f'{new_directory}/{folder_path}')
    else:
        print(f'Already Exists')

create_folder('Data/Indices')

Already Exists


In [5]:
Iran_Exports = pd.read_excel(f'{my_wd}/Data/Indices/Trademap_IranExports_2022.xlsx')
Iran_Exports['Code'] = Iran_Exports['Code'].str.replace("'","").astype('int')
#Iran_Exports['Code'] = Iran_Exports['Code'].astype("int")

remaining_columns = Iran_Exports.columns[2:]


remaining_columns=remaining_columns.str.replace("Exported value in ","")

Iran_Exports.columns=Iran_Exports.columns.str.replace("Exported value in ","")
Iran_Exports['HS4'] = Iran_Exports['Code']//100

Iran_Exports = Iran_Exports[(Iran_Exports['HS4']!=9999) & (Iran_Exports['HS4']!=2709) & (Iran_Exports['HS4']!=2710) & (Iran_Exports['HS4']!=2711) ]

Iran_Exports.columns = Iran_Exports.columns[0:2].tolist() + [col + '_Iran_Exports' for col in Iran_Exports.columns[2:]]

In [6]:
new_columns = [f'Share_{year}' for year in Iran_Exports.columns[2:]]
Share_Iran = pd.DataFrame(columns=['Code'] + new_columns)
Share_Iran['Code']=Iran_Exports['Code']

for year in range(2003,2023):
    Share_Iran[f'Share_{year}_Iran_Exports']=Iran_Exports[f'{year}_Iran_Exports']/(Iran_Exports[f'{year}_Iran_Exports'].sum())   


In [7]:
Countries = ['China','Turkey','Russia','SKorea','Iraq', 'Indonasia']
Iran_TCI=pd.DataFrame({"Country": Countries})

for country in Countries:
    Country_Imports=f'{country}_Imports'
    Share_Country=f'Share_{country}'
    Share_Country_Iran_Merged=f'Share_{country}_Iran_Merged'
    Share_diffs= f'{country}_Share_diff'
    print(Country_Imports)

    globals()[Country_Imports]=pd.read_excel(f'{my_wd}/Data/Indices/Trademap_{country}Imports_2023.xlsx')
    
    globals()[Country_Imports]['Code'] = globals()[Country_Imports]['Code'].str.replace("'","").astype('int')
    #globals()[Country_Imports]['Code'] = globals()[Country_Imports]['Code'].astype("int")
    
    remaining_columns = globals()[Country_Imports].columns[1:]
    remaining_columns=remaining_columns.str.replace("Imported value in ","")

    globals()[Country_Imports].columns=globals()[Country_Imports].columns.str.replace("Imported value in ","")


    globals()[Country_Imports]['HS4'] = globals()[Country_Imports]['Code']//100

    globals()[Country_Imports] = globals()[Country_Imports][(globals()[Country_Imports]['HS4']!=9999) &
                                                             (globals()[Country_Imports]['HS4']!=2709) &
                                                             (globals()[Country_Imports]['HS4']!=92710) &
                                                            (globals()[Country_Imports]['HS4']!=2711)]

    globals()[Country_Imports] = globals()[Country_Imports][globals()[Country_Imports]['Code']!=999999]

    globals()[Country_Imports].columns = globals()[Country_Imports].columns[0:1].tolist() + [col + f'_{country}_Imports' for col in globals()[Country_Imports].columns[1:]]

    new_columns = [f'Share_{year}' for year in globals()[Country_Imports].columns[1:]]
    globals()[Share_Country] = pd.DataFrame(columns=['Code'] + new_columns)
    globals()[Share_Country]['Code']=globals()[Country_Imports]['Code']

    globals()[Share_diffs]=pd.DataFrame()

    if country != "Iraq":

        for year in range(2004,2023):
            globals()[Share_Country][f'Share_{year}_{country}_Imports']=globals()[Country_Imports][f'{year}_{country}_Imports']/(globals()[Country_Imports][f'{year}_{country}_Imports'].sum()) 
        
        globals()[Share_Country_Iran_Merged]=Share_Iran.merge(globals()[Share_Country],on=['Code'],how='outer')
        #.fillna(0)

        globals()[Share_diffs]['Code']= globals()[Share_Country_Iran_Merged]['Code']

        for year in range (2009,2023):
            
            globals()[Share_diffs][f'diff_{year}']=abs(globals()[Share_Country_Iran_Merged][f'Share_{year}_{country}_Imports']-globals()[Share_Country_Iran_Merged][f'Share_{year}_Iran_Exports'])/2

            TCI_Year=100*(1-((globals()[Share_diffs][f'diff_{year}'].sum())))

            Iran_TCI.loc[Iran_TCI['Country']==country,f'{year}']=TCI_Year
        


    else:
        for year in range(2014,2023):
            globals()[Share_Country][f'Share_{year}_{country}_Imports']=globals()[Country_Imports][f'{year}_{country}_Imports']/(globals()[Country_Imports][f'{year}_{country}_Imports'].sum()) 
        
        globals()[Share_Country_Iran_Merged]=Share_Iran.merge(globals()[Share_Country],on=['Code'],how='outer')
        #.fillna(0)

        globals()[Share_diffs]['Code']= globals()[Share_Country_Iran_Merged]['Code']

        for year in range (2014,2023):
            
            globals()[Share_diffs][f'diff_{year}']=abs(globals()[Share_Country_Iran_Merged][f'Share_{year}_{country}_Imports']-globals()[Share_Country_Iran_Merged][f'Share_{year}_Iran_Exports'])/2
            
            TCI_Year=100*(1-((globals()[Share_diffs][f'diff_{year}'].sum())))

            Iran_TCI.loc[Iran_TCI['Country']==country,f'{year}']=TCI_Year

China_Imports
Turkey_Imports
Russia_Imports
SKorea_Imports
Iraq_Imports
Indonasia_Imports


In [9]:
Iran_TCI.to_excel(f'{my_wd}\Data\Indices\Iran_TCI_Oilless.xlsx',index=False)
#Iran_TCI